In [2]:
import asyncio

from src.utilities.get_stock_data import (
    get_stock_market_data,
    get_industry_stock_mapping_data,
)

In [ ]:
df1, df2 = await asyncio.gather(
    get_stock_market_data(), 
    get_industry_stock_mapping_data()
)

In [4]:
import asyncio

from src.utilities.get_stock_data_deprecated import (
    get_stock_market_data,
    get_industry_stock_mapping_data,
)

ImportError: cannot import name 'get_stock_market_data' from 'src.utilities.get_stock_data_deprecated' (/Users/zacks/Git/GitHub/value_investing/China/src/utilities/get_stock_data_deprecated.py)

In [ ]:
df1, df2 = get_stock_market_data(), get_industry_stock_mapping_data()